In [1]:
import numpy as np
import sys
sys.path.append('../../package/utility/')
sys.path.append('../../package/openmanipulator/')
from timer import Timer
from dynamixel_sdk import *
from motor import *
from pos_utils import *


In [2]:
data = np.load('./result/2024021071_timestep_and_qpos.npz')
timestep = data['timestep']
qpos = data['qpos']
print(f"timestep shape: {timestep.shape}, qpos shape: {qpos.shape}")

timestep shape: (6241, 1), qpos shape: (6241, 5)


In [3]:
qpos

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.24000000e-01],
       [-1.49457967e-22, -1.59864847e-20,  1.30636758e-23,
        -8.78327563e-23,  2.24000000e-01],
       [-9.62360258e-21, -7.34255249e-19,  8.74400250e-22,
        -3.98574711e-21,  2.24000000e-01],
       ...,
       [ 8.18457675e-06, -4.12222265e-04,  2.33479013e-03,
        -3.51409020e-03,  2.24000000e-01],
       [ 2.66937596e-06, -4.21621119e-04,  2.32902893e-03,
        -3.51634718e-03,  2.24000000e-01],
       [ 2.66937596e-06, -4.21621119e-04,  2.32902893e-03,
        -3.51634718e-03,  2.24000000e-01]])

In [4]:
# velocity와 acceleration 계산
dt = np.diff(timestep, axis=0).flatten()  # (n-1,)
velocity = np.diff(qpos, axis=0) / dt[:, None]  # (n-1, 5)
acceleration = np.diff(velocity, axis=0) / dt[:-1, None]  # (n-2, 5)


# residual_torque_limits = np.array(
#     [4.07267981906041,
#     3.95198067018901,
#     3.97675265052986,
#     3.96028660022148,
#     3.9
#     ])
vel_limits = np.array(
    [4.78501013007516,
    4.64320014851508,
    4.67230486141645,
    4.6529588236653,
    4.6])
acc_limits = np.array(
    [231321.273714306,
    16418.1218822564,
    160356.382291855,
    23470.8936359572,
    20000.0])
# 제한 초과 여부 확인
vel_exceed = np.abs(velocity) > vel_limits
acc_exceed = np.abs(acceleration) > acc_limits

# 결과 출력
print("Velocity limit exceeded at indices (row, joint):", np.argwhere(vel_exceed))
print("Acceleration limit exceeded at indices (row, joint):", np.argwhere(acc_exceed))

Velocity limit exceeded at indices (row, joint): []
Acceleration limit exceeded at indices (row, joint): []


In [5]:
if np.argwhere(vel_exceed).size == 0 and np.argwhere(acc_exceed).size == 0:
    tmr_cmd = Timer(_name='Command',_HZ=100,_MAX_SEC=np.inf,_VERBOSE=True)
    controller = XM430()
    controller.DXL_ID = [11,12,13,14,15]
    print("controller.DEVICENAME: ",controller.DEVICENAME)
    controller.BAUDRATE = 1000000
    controller.initialize()
    tmr_cmd.start()
    idx=0
    while idx<len(qpos):
        try:
            if tmr_cmd.do_run():
                qpos_bytes = rads_to_bytes(
                    qpos[idx],
                    [[-np.pi,np.pi]],
                )
                controller.run_multi_motor(qpos_bytes,controller.DXL_ID)
                idx += 1
        except KeyboardInterrupt:
            break

controller.DEVICENAME:  /dev/ttyUSB0
Succeeded to open the port
Succeeded to change the baudrate
[Command] start ([100]HZ. MAX_SEC:[infsec]).
